In [2]:
import warnings # 경고 메시지를 출력하지 않게 하기 위해 improt
warnings.filterwarnings("ignore") # 경고 메시지를 출력하지 않게 한다.
import numpy as np # 수치 계산, 배열 처리를 위해 import 한다.
import pandas as pd # 데이터 분석을 위한 전처리를 위해 import 한다.
import matplotlib as mpl # matplotlib에서 음수 데이터의 '-' 부호가 깨지는 것을 방지하기 위해 import한다.
mpl.rcParams['axes.unicode_minus'] = False # matplotlib에서 음수 데이터의 '-' 부호가 깨지는 것을 방지한다.
import matplotlib.pyplot as plt # 시각화를 하기 위해 import한다.
plt.rcParams["font.family"] = "Pretendard" # matplotlib에서 사용할 글꼴을 지정한다.
plt.rcParams["font.size"] = 10 # matplotlib에서 사용할 글꼴 크기를 지정한다.
import seaborn as sns # 시각화를 하기 위해 import한다. matplotlib 기반으로 만든 시각화 도구
from plotnine import * # R의 ggplot2 패키지를 사용하는 것처럼 만드는 시각화 도구를 import한다.
import missingno as msno # 누락값의 분포를 그래프 형태로 표현한다.
import folium
# import requests
# import json
# from pandas.io.json import json_normalize
# from bs4 import BeautifulSoup

In [ ]:
'''
팀 프로젝트의 연장선
전국 6대 편의점 주소 정보를 웹 크롤링으로 가져왔음. 현재 지도 정보 4만개 이상 => 총 6만개 이상 추정
데이터를 최대한 정제하기는 했으나, 모든 데이터를 확인하는 것은 무리가 있고 자바 내에서 일일이 바꾸는 것은 한계가 있고, 시각화를 편하게 하기 위해 분석 쪽은 개인 프로젝트로 넘기게 되었음.
실제로 확인해 보니 중복 데이터나 오타, 띄어쓰기가 안 되어 있는 자료 등이 상당수 확인되었음.
'''

In [4]:
whereCVS =  pd.read_csv('./data/wherecvs.csv', encoding='cp949')

In [21]:
whereCVS['시도'] = whereCVS['ADDRESS'].str.split(' ').str[0]
whereCVS['구군'] = whereCVS['ADDRESS'].str.split(' ').str[1]
whereCVS.to_csv('./output/newWhereCVS.csv')
whereCVS

,MAPIDX,STORENAME,WHICHCVS,ADDRESS,시도,구군
0,1,우리들북구청점,C·SPACE,울산 북구 무룡로 29 (연암동),울산,북구
1,2,초읍점,C·SPACE,부산 부산진구 월드컵대로474번길 23 (초읍동),부산,부산진구
2,3,부산송도점,C·SPACE,부산 서구 충무대로 80 (암남동),부산,서구
3,4,서생점,C·SPACE,울산 울주군 서생면 나사해안길 5-16,울산,울주군
4,5,꽃바위점,C·SPACE,"울산 동구 문현1길 37, 상가동1호(방어동, 정림아파트)",울산,동구
...,...,...,...,...,...,...
46157,46158,충주호암점,세븐일레븐,충북 충주시 호암중앙2로 32 101호(호암동),충북,충주시
46158,46159,충주호암LH점,세븐일레븐,충북 충주시 호암수청2로 56,충북,충주시
46159,46160,충주힐스테이트점,세븐일레븐,"충북 충주시 호암수청1로 29 (호암동, LH힐스테이트)",충북,충주시
46160,46161,충청대혁신점,세븐일레븐,충북 청주시 흥덕구 강내면 탑연월곡길 62-15,충북,청주시


In [27]:
whereCVS['시도'].value_counts().tail(10)

경남창원시의창구반계로98        1
경남창원시성산구창이대로692번길    1
경남창원시성산구             1
부선광역시                1
부산시                  1
용인시                  1
대구시                  1
아산시                  1
포천시                  1
대구달성군현풍면현풍중아로        1
Name: 시도, dtype: int64

In [3]:
newWhereCVS =  pd.read_csv('./output/newWhereCVS.csv', encoding='cp949')
newWhereCVS.columns.name = 'MAPIDX'
newWhereCVS

MAPIDX,STORENAME,WHICHCVS,ADDRESS,시도,구군
0,횡성역점,C·SPACE,강원 횡성군 횡성읍 덕고로 591,강원도,횡성군
1,춘천점,C·SPACE,강원 춘천시 동내면 순환대로 716,강원도,춘천시
2,횡성시니어클럽6호점,C·SPACE,강원 횡성군 횡성읍 대동로 8,강원도,횡성군
3,원주기업도시점,C·SPACE,강원 원주시 지정면 기업도시로 125,강원도,원주시
4,현무점,C·SPACE,강원 춘천시 남산면 북한강변길 378,강원도,춘천시
...,...,...,...,...,...
46157,현대대소산단점,CU,충청북도 음성군 대소면 대금로 199 (오류리),충청북도,음성군
46158,현대오일뱅크문의IC점,CU,충청북도 청주시 상당구 문의면 미천고은로 61 (남계리),충청북도,청주시
46159,현대오일뱅크서오창IC,CU,충청북도 청주시 청원구 오창읍 오창대로 149 구룡셀프주유소 (구룡리) 구룡셀프주유소,충청북도,청주시
46160,현대원각주유소점,CU,충청북도 옥천군 동이면 옥천로 2058 (세산리),충청북도,옥천군


In [8]:
from geopy.geocoders import Nominatim
def geocoding(address):
    geolocoder = Nominatim(user_agent='South Korea', timeout=None)
    geo = geolocoder.geocode(address)
    return [geo.latitude, geo.longitude]

newWhereCVS['위도'] = 0.0
newWhereCVS['경도'] = 0.0
newWhereCVS

MAPIDX,STORENAME,WHICHCVS,ADDRESS,시도,구군,위도,경도
0,횡성역점,C·SPACE,강원 횡성군 횡성읍 덕고로 591,강원도,횡성군,0.0,0.0
1,춘천점,C·SPACE,강원 춘천시 동내면 순환대로 716,강원도,춘천시,0.0,0.0
2,횡성시니어클럽6호점,C·SPACE,강원 횡성군 횡성읍 대동로 8,강원도,횡성군,0.0,0.0
3,원주기업도시점,C·SPACE,강원 원주시 지정면 기업도시로 125,강원도,원주시,0.0,0.0
4,현무점,C·SPACE,강원 춘천시 남산면 북한강변길 378,강원도,춘천시,0.0,0.0
...,...,...,...,...,...,...,...
46157,현대대소산단점,CU,충청북도 음성군 대소면 대금로 199 (오류리),충청북도,음성군,0.0,0.0
46158,현대오일뱅크문의IC점,CU,충청북도 청주시 상당구 문의면 미천고은로 61 (남계리),충청북도,청주시,0.0,0.0
46159,현대오일뱅크서오창IC,CU,충청북도 청주시 청원구 오창읍 오창대로 149 구룡셀프주유소 (구룡리) 구룡셀프주유소,충청북도,청주시,0.0,0.0
46160,현대원각주유소점,CU,충청북도 옥천군 동이면 옥천로 2058 (세산리),충청북도,옥천군,0.0,0.0


In [9]:
for i in range (0, len(newWhereCVS['ADDRESS'])-1):
    try:
        newWhereCVS['위도'][i] = geocoding(newWhereCVS['ADDRESS'][i])[0]
        newWhereCVS['경도'][i] = geocoding(newWhereCVS['ADDRESS'][i])[1]    
        print(i)    
    except:
        newWhereCVS['위도'][i] = 0
        newWhereCVS['위도'][i] = 0
        continue

0
1
2
3
6


In [ ]:
cvs_map = folium.Map(location=[newWhereCVS['위도'].mean(), newWhereCVS['경도'].mean()], zoom_start=12)

for index, data in newWhereCVS.iterrows():
    string = '[{}] {}'.format(data['WHICHCVS'], data['STORENAME'])
    popup = folium.Popup(string, max_width=300)
    folium.Marker(location=[data['위도'], data['경도']], popup=popup).add_to(cvs_map)

cvs_map.save('./output/cvs_map.html')
cvs_map
